<a href="https://colab.research.google.com/github/JavZon/Course_WM/blob/main/Stacking_AML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import accuracy_score, classification_report, f1_score

Importing Stacking Classifier

In [2]:
from sklearn.ensemble import StackingClassifier

Importing dataset (classification)

 This dataset is about breast cancer. Here **M** means **malignant** (cancerous) and **B** means **benign** (non-cancerous)

In [4]:
df=pd.read_csv('/content/breast-cancer.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [6]:
df=df.drop(columns='id')

In [7]:
df.head(2)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902


In [8]:
df['diagnosis'].value_counts()

,count
diagnosis,
B,357
M,212


In [9]:
df['diagnosis']=df['diagnosis'].map({'B':0, 'M':1})

In [10]:
df.corrwith(df['diagnosis']).sort_values(ascending=False)

,0
diagnosis,1.000000
concave points_worst,0.793566
perimeter_worst,0.782914
concave points_mean,0.776614
radius_worst,0.776454
perimeter_mean,0.742636
area_worst,0.733825
radius_mean,0.730029
area_mean,0.708984
concavity_mean,0.696360


In [11]:
X=df.drop(columns='diagnosis')
y=df['diagnosis']

mut_info=mutual_info_classif(X,y)
mut_info

array([0.36714042, 0.09998632, 0.40345295, 0.36053562, 0.07844225,
       0.21318146, 0.37352788, 0.44198412, 0.07132382, 0.00866539,
       0.24653246, 0.        , 0.27622096, 0.34000957, 0.01757617,
       0.07490964, 0.11694241, 0.12661731, 0.01316511, 0.03950802,
       0.45585432, 0.12118086, 0.47485   , 0.46492495, 0.09535636,
       0.22534324, 0.31753992, 0.43689647, 0.09415576, 0.07187084])

In [12]:
corr_df = pd.DataFrame(
    {'Mut_info': mut_info, 'Correlation': df.corrwith(df['diagnosis'])},
    index=df.columns.drop('diagnosis'))
corr_df

,Mut_info,Correlation
radius_mean,0.367140,0.730029
texture_mean,0.099986,0.415185
perimeter_mean,0.403453,0.742636
area_mean,0.360536,0.708984
smoothness_mean,0.078442,0.358560
compactness_mean,0.213181,0.596534
concavity_mean,0.373528,0.696360
concave points_mean,0.441984,0.776614
symmetry_mean,0.071324,0.330499
fractal_dimension_mean,0.008665,-0.012838


At first we create models for each Models, then create stacked_model

In [13]:
# scaling and splitting dataset
X=df.drop(columns='diagnosis')
y=df['diagnosis'].copy()

# scaling
scaler=StandardScaler()
X=scaler.fit_transform(X)

#splitting
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)


(455, 30)
(114, 30)


**RF model**

In [15]:
# rf_model
rf_model=RandomForestClassifier()
rf_model.fit(X_train,y_train)

rf_pred=rf_model.predict(X_test)
acc_score=accuracy_score(rf_pred,y_test)
f1=f1_score(rf_pred, y_test)
class_rep=classification_report(rf_pred, y_test)
print(f'Accuracy score is -',acc_score)
print(f'F1 score is -',f1)
print(class_rep)

Accuracy score is - 0.9649122807017544
F1 score is - 0.9523809523809523
              precision    recall  f1-score   support

           0       0.99      0.96      0.97        73
           1       0.93      0.98      0.95        41

    accuracy                           0.96       114
   macro avg       0.96      0.97      0.96       114
weighted avg       0.97      0.96      0.97       114



**SVM model**

In [17]:
# svm model
from sklearn.svm import SVC
svm_model=SVC()
svm_model.fit(X_train,y_train)

svm_pred=svm_model.predict(X_test)
acc_score=accuracy_score(svm_pred,y_test)
f1=f1_score(svm_pred, y_test)
class_rep=classification_report(svm_pred, y_test)
print(f'Accuracy score is -',acc_score)
print(f'F1 score is -',f1)
print(class_rep)

Accuracy score is - 0.9736842105263158
F1 score is - 0.9647058823529412
              precision    recall  f1-score   support

           0       0.99      0.97      0.98        72
           1       0.95      0.98      0.96        42

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114



**LogisticRegression model**

In [18]:
logistic_model=LogisticRegression()
logistic_model.fit(X_train,y_train)

logistic_pred=svm_model.predict(X_test) # prediction
acc_score=accuracy_score(logistic_pred,y_test)
f1=f1_score(logistic_pred, y_test)
class_rep=classification_report(logistic_pred, y_test)
print(f'Accuracy score is -',acc_score)
print(f'F1 score is -',f1)
print(class_rep)

Accuracy score is - 0.9736842105263158
F1 score is - 0.9647058823529412
              precision    recall  f1-score   support

           0       0.99      0.97      0.98        72
           1       0.95      0.98      0.96        42

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114



Stacking method (Advanced ML)

In [20]:
# base models
base_models= [
    ('rf', RandomForestClassifier(random_state=42)),
    ('svm', SVC(random_state=42)) ]

# meta_model
meta_model=LogisticRegression()

In [21]:
# stacking model (classifier)
stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5  # Cross-validation for meta-model training
)

# Train stacking model
stacking_model.fit(X_train, y_train)

# Make predictions
predictions = stacking_model.predict(X_test)

# Evaluate
print(f"Stacking Accuracy: {accuracy_score(y_test, predictions):.2f}")



Stacking Accuracy: 0.96


In [22]:
class_rep=classification_report(predictions, y_test)
print(class_rep)

              precision    recall  f1-score   support

           0       0.97      0.97      0.97        71
           1       0.95      0.95      0.95        43

    accuracy                           0.96       114
   macro avg       0.96      0.96      0.96       114
weighted avg       0.96      0.96      0.96       114

